# Author Identification Using The Information Bottleneck Method

In [201]:
import pandas as pd
import os
import numpy as np
import re

In [202]:
word_list = []
author_list = []
number_of_authors = 0
file_idx = 0

for file in os.listdir('books/'):
    if file.endswith(".txt"):
        with open(os.path.join('books/'+file), 'r',encoding='utf-8') as f:
            for line in f:
                for word in line.split():
                    word1 = " ".join(re.findall('[a-zA-ZäöüßÄÖÜẞ]+', word))
                    if not word1 in word_list:
                        word_list.append(word1.lower())
        number_of_authors += 1
        author_list.append(file.split('_')[0])
        

p_x_y = np.zeros((word_list.__len__(),number_of_authors))
for file in os.listdir('books/'):
    if file.endswith(".txt"):
        with open(os.path.join('books/'+file), 'r',encoding='utf-8') as f:
            for line in f:
                for word in line.split():
                    word1 = " ".join(re.findall('[a-zA-ZäöüßÄÖÜẞ]+', word))
                    index = word_list.index(word1.lower())
                    p_x_y[index,file_idx] += 1
        file_idx += 1
        
p_x_y /= p_x_y.sum()
p_y = p_x_y.sum(axis=1)

In [203]:
np.argmax(p_x_y[:,1])

39

In [204]:
non_unique_indices = np.nonzero(np.sum(p_x_y!=0,axis=1)>1)[0]
p_y = p_y[non_unique_indices]
p_x_y = p_x_y[non_unique_indices,:]
#non_unique_indices
word_list = [word_list[j] for j in non_unique_indices]

In [205]:
word_table = pd.DataFrame({ 'Pr(Word)' : p_y,
                          'Word' : word_list})

In [217]:
word_table.sort_values(by='Pr(Word)', ascending=False)

,Pr(Word),Word
27,0.032978,und
4,0.024685,die
33,0.020155,der
12,0.015420,ich
68,0.013477,sie
20,0.013346,zu
90,0.012515,in
70,0.012216,nicht
236,0.012085,er
74,0.012011,das


In [213]:
from information_bottleneck.information_bottleneck_algorithms.KLmeansIB_class import KLmeansIB as KLmeans

card_T = 10
IB_inst = KLmeans(p_x_y+1e-34,
                   card_T,
                   nror=100)
IB_inst.run_IB_algo()

###  OPENCL Device #####
[<pyopencl.Device 'Intel(R) UHD Graphics 620' on 'Intel(R) OpenCL' at 0x2b404399dc0>]


 [Elapsed Time: 0:00:09] |##################################| (Time:  0:00:09) 


In [214]:
IB_inst.display_MIs()

----- Mutual Information Comp --- 
-----  KL means IB  ------ 
MI_XT_s=  0.18965189361022478
MI_XY_s=  0.4280667070931024
ratio=  0.4430428493215578


In [215]:
p_t_given_y, p_x_given_t, p_t = IB_inst.get_results()

In [216]:
cluster_ind = p_t_given_y.argmax(axis=1)
word_list = np.asarray(word_list)

words_in_cluster = [None]*card_T

for t in range(card_T):
    indices = np.argsort(p_y[np.argwhere(cluster_ind == t)][:, 0])
    words_in_cluster[t] = []
    for item in word_list[np.argwhere(cluster_ind == t)[indices]]:
        words_in_cluster[t].append(item[0])
    
result_dataframe = pd.DataFrame(
    {'cluster index t' : np.arange(card_T),
     'words in cluster': words_in_cluster
    }
)
for x in range(p_x_given_t.shape[1]):
    result_dataframe['p(X='+author_list[x]+'|t)'] = pd.Series(p_x_given_t[:,x])
result_dataframe

,cluster index t,words in cluster,p(X=goethe|t),p(X=kafka|t),p(X=mann|t)
0,0,"[flehen, spruch, sünde, sacht, suche, edle, li...",0.800816,0.046803,0.152381
1,1,"[innen, unanständige, schwanken, genähert, wär...",0.457520,0.456776,0.085704
2,2,"[lasse, rohen, deutschen, möchten, letztenmal,...",0.477121,0.242237,0.280642
3,3,"[gewendet, fernen, jünger, studien, feine, ver...",0.292243,0.420554,0.287203
4,4,"[schwindel, wangen, scheiben, erfüllt, bestand...",0.263556,0.324761,0.411683
5,5,"[ärgerlich, gebrauchen, knochen, ermahnungen, ...",0.026561,0.767008,0.206431
6,6,"[dunkel, personal, gelernt, bemerkt, entscheid...",0.091347,0.534890,0.373763
7,7,"[walten, schmecken, loben, zofe, hofe, geschwi...",0.334855,0.022222,0.642922
8,8,"[gerichtet, setzt, studiert, heller, sonntags,...",0.172524,0.292090,0.535386
9,9,"[lebhafter, empfand, erwies, tüchtigen, nachde...",0.003858,0.294367,0.701775
